<a href="https://colab.research.google.com/github/Vaishnavi-TCD/TrustLink/blob/main/GPT_ML_Week9_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64
block_size = 256
max_iters = 1000  # Reduced for faster experimentation
eval_interval = 200
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
dropout = 0.2
torch.manual_seed(1337)

file_path = '/content/input_childSpeech_trainingSet.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1)) if targets is not None else None
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# Downsize configurations |
configs = [
    {"n_embd": 128, "n_head": 2, "n_layer": 2},
    {"n_embd": 96, "n_head": 4, "n_layer": 5},
    {"n_embd": 128, "n_head": 3, "n_layer": 3},
    {"n_embd": 160, "n_head": 3, "n_layer": 3},
    {"n_embd": 144, "n_head": 2, "n_layer": 3},
    {"n_embd": 192, "n_head": 3, "n_layer": 3},
    {"n_embd": 256, "n_head": 2, "n_layer": 2},
    {"n_embd": 128, "n_head": 4, "n_layer": 4},
    {"n_embd": 160, "n_head": 4, "n_layer": 3},
    {"n_embd": 192, "n_head": 4, "n_layer": 2},
]

for i, config in enumerate(configs):
    n_embd = config["n_embd"]
    n_head = config["n_head"]
    n_layer = config["n_layer"]

    model = GPTLanguageModel()
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    param_count = sum(p.numel() for p in model.parameters())
    print(f"Configuration {i + 1}: {config}")
    print(f"Total Parameters: {param_count / 1e6:.6f} M")

    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        xb, yb = get_batch('train')
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    generated_text = decode(model.generate(context, max_new_tokens=500)[0].tolist())
    file_name = f"generated_text_config_{i + 1}_params_{param_count}.txt"
    with open(file_name, "w") as f:
        f.write(generated_text)
    print(f"Generated text saved to {file_name}")


Configuration 1: {'n_embd': 128, 'n_head': 2, 'n_layer': 2}
Total Parameters: 0.439080 M
step 0: train loss 3.7846, val loss 3.7845
step 200: train loss 1.4985, val loss 1.5101
step 400: train loss 0.3816, val loss 0.3850
step 600: train loss 0.3592, val loss 0.3619
step 800: train loss 0.3515, val loss 0.3538
step 999: train loss 0.3416, val loss 0.3448
Generated text saved to generated_text_config_1_params_439080.txt
Configuration 2: {'n_embd': 192, 'n_head': 2, 'n_layer': 2}
Total Parameters: 0.953512 M
step 0: train loss 3.8148, val loss 3.8139
step 200: train loss 0.6386, val loss 0.6455
step 400: train loss 0.3684, val loss 0.3711
step 600: train loss 0.3525, val loss 0.3558
step 800: train loss 0.3454, val loss 0.3500
step 999: train loss 0.3402, val loss 0.3434
Generated text saved to generated_text_config_2_params_953512.txt
Configuration 3: {'n_embd': 128, 'n_head': 3, 'n_layer': 3}
Total Parameters: 0.633896 M
step 0: train loss 3.6740, val loss 3.6727
step 200: train loss 1

In [ ]:
# with the bias included
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64
block_size = 256
max_iters = 1000  # Reduced for faster experimentation
eval_interval = 200
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
dropout = 0.2
torch.manual_seed(1337)


file_path = '/content/input_childSpeech_trainingSet.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()


chars = sorted(list(set(text)))
vocab_size = len(chars)


stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=True)
        self.query = nn.Linear(n_embd, head_size, bias=True)
        self.value = nn.Linear(n_embd, head_size, bias=True)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1)) if targets is not None else None
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

configs = [
    {"n_embd": 96, "n_layer": 5, "n_head": 4},
    {"n_embd": 160, "n_head": 4, "n_layer": 3},  # Balanced size
    {"n_embd": 192, "n_head": 4, "n_layer": 2},  # Mid-size
]


for i, config in enumerate(configs):
    n_embd = config["n_embd"]
    n_head = config["n_head"]
    n_layer = config["n_layer"]

    model = GPTLanguageModel()
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    param_count = sum(p.numel() for p in model.parameters())
    print(f"Configuration {i + 1}: {config}")
    print(f"Total Parameters: {param_count / 1e6:.6f} M")

    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        xb, yb = get_batch('train')
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    generated_text = decode(model.generate(context, max_new_tokens=500)[0].tolist())
    file_name = f"generated_text_config_{i + 1}_params_{param_count}.txt"
    with open(file_name, "w") as f:
        f.write(generated_text)
    print(f"Generated text saved to {file_name}")


Configuration 1: {'n_embd': 96, 'n_layer': 5, 'n_head': 4}
Total Parameters: 0.591688 M
step 0: train loss 3.7445, val loss 3.7435
step 200: train loss 1.8876, val loss 1.8971
step 400: train loss 0.4095, val loss 0.4124
step 600: train loss 0.3618, val loss 0.3630
step 800: train loss 0.3479, val loss 0.3504
step 999: train loss 0.3456, val loss 0.3477
Generated text saved to generated_text_config_1_params_591688.txt
Configuration 2: {'n_embd': 160, 'n_head': 4, 'n_layer': 3}
Total Parameters: 0.981960 M
step 0: train loss 3.7202, val loss 3.7201
step 200: train loss 1.1862, val loss 1.1960
step 400: train loss 0.3579, val loss 0.3618
step 600: train loss 0.3436, val loss 0.3468
step 800: train loss 0.3405, val loss 0.3430
step 999: train loss 0.3382, val loss 0.3424
Generated text saved to generated_text_config_2_params_981960.txt
Configuration 3: {'n_embd': 192, 'n_head': 4, 'n_layer': 2}
Total Parameters: 0.954664 M
step 0: train loss 3.7428, val loss 3.7429
step 200: train loss 0.

In [ ]:
# without the bias and without skip connection
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64
block_size = 256
max_iters = 1000  # Reduced for faster experimentation
eval_interval = 200
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
dropout = 0.2
torch.manual_seed(1337)


file_path = '/content/input_childSpeech_trainingSet.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()


chars = sorted(list(set(text)))
vocab_size = len(chars)


stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # x = x + self.sa(self.ln1(x))
        # x = x + self.ffwd(self.ln2(x))
        x = self.sa(self.ln1(x))
        x = self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1)) if targets is not None else None
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

configs = [

    {"n_embd": 96, "n_layer": 5, "n_head": 4},
    {"n_embd": 160, "n_head": 4, "n_layer": 3},  # Balanced size
    {"n_embd": 192, "n_head": 4, "n_layer": 2},  # Mid-size
]


for i, config in enumerate(configs):
    n_embd = config["n_embd"]
    n_head = config["n_head"]
    n_layer = config["n_layer"]

    model = GPTLanguageModel()
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    param_count = sum(p.numel() for p in model.parameters())
    print(f"Configuration {i + 1}: {config}")
    print(f"Total Parameters: {param_count / 1e6:.6f} M")

    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        xb, yb = get_batch('train')
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()


    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    generated_text = decode(model.generate(context, max_new_tokens=500)[0].tolist())
    file_name = f"generated_text_config_without_skip{i + 1}_params_{param_count}.txt"
    with open(file_name, "w") as f:
        f.write(generated_text)
    print(f"Generated text saved to {file_name}")


Configuration 1: {'n_embd': 96, 'n_layer': 5, 'n_head': 4}
Total Parameters: 0.590248 M
step 0: train loss 3.6844, val loss 3.6832
step 200: train loss 3.1274, val loss 3.1295
step 400: train loss 3.1274, val loss 3.1299
step 600: train loss 3.1269, val loss 3.1299
step 800: train loss 3.1278, val loss 3.1303
step 999: train loss 3.1265, val loss 3.1295
Generated text saved to generated_text_config_without_skip1_params_590248.txt
Configuration 2: {'n_embd': 160, 'n_head': 4, 'n_layer': 3}
Total Parameters: 0.980520 M
step 0: train loss 3.7470, val loss 3.7482
step 200: train loss 3.1286, val loss 3.1303
step 400: train loss 3.1286, val loss 3.1303
step 600: train loss 3.1275, val loss 3.1302
step 800: train loss 3.1275, val loss 3.1301
step 999: train loss 3.1276, val loss 3.1300
Generated text saved to generated_text_config_without_skip2_params_980520.txt
Configuration 3: {'n_embd': 192, 'n_head': 4, 'n_layer': 2}
Total Parameters: 0.953512 M
step 0: train loss 3.7240, val loss 3.7258

In [ ]:
# Part 2
# considering configuraton 10 n_embed = 192, n_head = 4, n_layer = 2
# with no bias and with skip connections
# train the model of training dataset and test the model on test and shakespeare dataset
# Includes baseline model and it's corresponding dataset's losses as well
# the character list include unque characters from all datasets

import torch
import torch.nn as nn
from torch.nn import functional as F

# Configuration 10 parameters
n_embd = 192
n_head = 4
n_layer = 2
batch_size = 64
block_size = 256
max_iters = 1000
eval_interval = 200
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dropout = 0.2
torch.manual_seed(1337)

# Dataset path
file_path_train = '/content/input_childSpeech_trainingSet.txt'
file_path_child_test = '/content/input_childSpeech_testSet.txt'
file_path_shakespeare_test = '/content/input_shakespeare.txt'

# Load datasets
with open(file_path_train, 'r', encoding='utf-8') as f:
    train_text = f.read()

with open(file_path_child_test, 'r', encoding='utf-8') as f:
    test_child_text = f.read()

with open(file_path_shakespeare_test, 'r', encoding='utf-8') as f:
    test_shakespeare_text = f.read()

# Unique characters and vocab size
all_text = train_text + test_child_text + test_shakespeare_text
chars = sorted(list(set(all_text)))
vocab_size = len(chars)

# Mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Encode datasets
train_data = torch.tensor(encode(train_text), dtype=torch.long)
test_child_data = torch.tensor(encode(test_child_text), dtype=torch.long)
test_shakespeare_data = torch.tensor(encode(test_shakespeare_text), dtype=torch.long)

# Split train/validation
n = int(0.9 * len(train_data))
train_data, val_data = train_data[:n], train_data[n:]

# Data loading function
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Estimate loss
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Evaluate test loss
@torch.no_grad()
def evaluate_test_loss(data):
    model.eval()
    total_loss = 0
    count = 0
    for i in range(0, len(data) - block_size, block_size):
        x = data[i:i+block_size].unsqueeze(0).to(device)
        y = data[i+1:i+block_size+1].unsqueeze(0).to(device)
        _, loss = model(x, y)
        total_loss += loss.item()
        count += 1
    return total_loss / count

# Baseline model (dummy)
@torch.no_grad()
def evaluate_baseline_loss(data):
    baseline_loss = 0
    count = 0
    for i in range(0, len(data) - block_size, block_size):
        y = data[i + 1:i + block_size + 1].to(device)  # Target data
        y = y.view(-1)  # Flatten target to match input shape
        # Create uniform probabilities matching the length of the target
        probs = torch.ones((len(y), vocab_size), device=device) / vocab_size
        # Calculate cross-entropy loss
        loss = F.cross_entropy(probs, y)
        baseline_loss += loss.item()
        count += 1
    return baseline_loss / count


# Model components
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=True)
        self.query = nn.Linear(n_embd, head_size, bias=True)
        self.value = nn.Linear(n_embd, head_size, bias=True)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1)) if targets is not None else None
        return logits, loss

# Train Configuration 10
model = GPTLanguageModel()
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

print("Training Configuration 10...")
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Evaluate on test datasets
test_child_loss = evaluate_test_loss(test_child_data)
test_shakespeare_loss = evaluate_test_loss(test_shakespeare_data)

print(f"Test Loss on input_childSpeech_testSet.txt: {test_child_loss:.4f}")
print(f"Test Loss on input_shakespeare.txt: {test_shakespeare_loss:.4f}")

# Evaluate baseline loss
baseline_child_loss = evaluate_baseline_loss(test_child_data)
baseline_shakespeare_loss = evaluate_baseline_loss(test_shakespeare_data)

print(f"Baseline Loss on input_childSpeech_testSet.txt: {baseline_child_loss:.4f}")
print(f"Baseline Loss on input_shakespeare.txt: {baseline_shakespeare_loss:.4f}")


Training Configuration 10...
step 0: train loss 4.2066, val loss 4.2066
step 200: train loss 0.8365, val loss 0.8442
step 400: train loss 0.3603, val loss 0.3654
step 600: train loss 0.3447, val loss 0.3492
step 800: train loss 0.3408, val loss 0.3448
step 999: train loss 0.3362, val loss 0.3410
Test Loss on input_childSpeech_testSet.txt: 0.3489
Test Loss on input_shakespeare.txt: 6.5660
Baseline Loss on input_childSpeech_testSet.txt: 4.1744
Baseline Loss on input_shakespeare.txt: 4.1744


In [ ]:
# Part 2
# considering configuraton 10 n_embed = 192, n_head = 4, n_layer = 2
# with no bias and with skip connections
# train the model of training dataset and test the model on test and shakespeare dataset
# Includes baseline model and it's corresponding dataset's losses as well
# the character list include unique characters from all datasets

import torch
import torch.nn as nn
from torch.nn import functional as F
# hyper paramters as per the configuration 10
n_embd = 192
n_head = 4
n_layer = 2
batch_size = 64
block_size = 256
max_iters = 1000
eval_interval = 200
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dropout = 0.2
torch.manual_seed(1337)


file_path_train = '/content/input_childSpeech_trainingSet.txt'
file_path_child_test = '/content/input_childSpeech_testSet.txt'
file_path_shakespeare_test = '/content/input_shakespeare.txt'


with open(file_path_train, 'r', encoding='utf-8') as f:
    train_text = f.read()

with open(file_path_child_test, 'r', encoding='utf-8') as f:
    test_child_text = f.read()

with open(file_path_shakespeare_test, 'r', encoding='utf-8') as f:
    test_shakespeare_text = f.read()


all_text = train_text + test_child_text + test_shakespeare_text
chars = sorted(list(set(all_text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

train_data = torch.tensor(encode(train_text), dtype=torch.long)
test_child_data = torch.tensor(encode(test_child_text), dtype=torch.long)
test_shakespeare_data = torch.tensor(encode(test_shakespeare_text), dtype=torch.long)
n = int(0.9 * len(train_data))
train_data, val_data = train_data[:n], train_data[n:]


def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

@torch.no_grad()
def evaluate_test_loss(data):
    model.eval()
    total_loss = 0
    count = 0
    for i in range(0, len(data) - block_size, block_size):
        x = data[i:i+block_size].unsqueeze(0).to(device)
        y = data[i+1:i+block_size+1].unsqueeze(0).to(device)
        _, loss = model(x, y)
        total_loss += loss.item()
        count += 1
    return total_loss / count

# Baseline model |
@torch.no_grad()
def evaluate_baseline_loss(data):
    baseline_loss = 0
    count = 0
    for i in range(0, len(data) - block_size, block_size):
        y = data[i + 1:i + block_size + 1].to(device)
        y = y.view(-1)
        probs = torch.ones((len(y), vocab_size), device=device) / vocab_size
        loss = F.cross_entropy(probs, y)
        baseline_loss += loss.item()
        count += 1
    return baseline_loss / count


class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=True)
        self.query = nn.Linear(n_embd, head_size, bias=True)
        self.value = nn.Linear(n_embd, head_size, bias=True)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1)) if targets is not None else None
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx



@torch.no_grad()
def generate_predictions(model, data, output_file, max_tokens=500):
    model.eval()
    context = data[:block_size].unsqueeze(0).to(device)
    generated = model.generate(context, max_new_tokens=max_tokens)
    generated_text = decode(generated[0].tolist())


    with open(output_file, "w") as f:
        f.write(generated_text)
    print(f"Generated predictions saved to {output_file}")

model = GPTLanguageModel()
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

print("Training Configuration 10...")
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

test_child_loss = evaluate_test_loss(test_child_data)
test_shakespeare_loss = evaluate_test_loss(test_shakespeare_data)

print(f"Test Loss on input_childSpeech_testSet.txt: {test_child_loss:.4f}")
print(f"Test Loss on input_shakespeare.txt: {test_shakespeare_loss:.4f}")

baseline_child_loss = evaluate_baseline_loss(test_child_data)
baseline_shakespeare_loss = evaluate_baseline_loss(test_shakespeare_data)

print(f"Baseline Loss on input_childSpeech_testSet.txt: {baseline_child_loss:.4f}")
print(f"Baseline Loss on input_shakespeare.txt: {baseline_shakespeare_loss:.4f}")

generate_predictions(model, test_child_data, "predictions_childSpeech.txt", max_tokens=500)
generate_predictions(model, test_shakespeare_data, "predictions_shakespeare.txt", max_tokens=500)


Training Configuration 10...
step 0: train loss 4.2066, val loss 4.2066
step 200: train loss 0.8365, val loss 0.8442
step 400: train loss 0.3603, val loss 0.3654
step 600: train loss 0.3447, val loss 0.3492
step 800: train loss 0.3408, val loss 0.3448
step 999: train loss 0.3362, val loss 0.3410
Test Loss on input_childSpeech_testSet.txt: 0.3489
Test Loss on input_shakespeare.txt: 6.5660
Baseline Loss on input_childSpeech_testSet.txt: 4.1744
Baseline Loss on input_shakespeare.txt: 4.1744
Generated predictions saved to predictions_childSpeech.txt
Generated predictions saved to predictions_shakespeare.txt


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters (template)
batch_size = 64
block_size = 256
max_iters = 1000  # Reduced for faster experimentation
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
dropout = 0.2
torch.manual_seed(1337)

# Dataset path
file_path = '/content/input_childSpeech_trainingSet.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

# Unique characters and vocab size
chars = sorted(list(set(text)))
vocab_size = len(chars)

# Create mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Encode dataset and split into train/val
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# Data loading function
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# Estimate loss
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Model components
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1)) if targets is not None else None
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# Downsize configurations
configs = [
    {"n_embd": 128, "n_head": 2, "n_layer": 2},  # Smaller size, ~500K params
    {"n_embd": 192, "n_head": 2, "n_layer": 2},  # Mid-size, closer to ~700K params
    {"n_embd": 128, "n_head": 3, "n_layer": 3},  # Balanced, ~600-800K params
    {"n_embd": 160, "n_head": 3, "n_layer": 3},  # Mid-size, closer to ~800K params
    {"n_embd": 144, "n_head": 2, "n_layer": 3},  # ~600K params
    {"n_embd": 192, "n_head": 3, "n_layer": 3},  # Slightly larger, ~900K params
    {"n_embd": 256, "n_head": 2, "n_layer": 2},  # Larger embedding but smaller layers/heads
    {"n_embd": 128, "n_head": 4, "n_layer": 4},  # Near the upper limit ~1M params
    {"n_embd": 160, "n_head": 4, "n_layer": 3},  # Balanced size
    {"n_embd": 192, "n_head": 4, "n_layer": 2},  # Mid-size
]


# Train models for each configuration and save generated text
for i, config in enumerate(configs):
    n_embd = config["n_embd"]
    n_head = config["n_head"]
    n_layer = config["n_layer"]

    model = GPTLanguageModel()
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    param_count = sum(p.numel() for p in model.parameters())
    print(f"Configuration {i + 1}: {config}")
    print(f"Total Parameters: {param_count / 1e6:.6f} M")

    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        xb, yb = get_batch('train')
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    # Generate text and save to a file
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    generated_text = decode(model.generate(context, max_new_tokens=500)[0].tolist())
    file_name = f"generated_text_config_{i + 1}_params_{param_count}.txt"
    with open(file_name, "w") as f:
        f.write(generated_text)
    print(f"Generated text saved to {file_name}")

    # Download file to local (optional, for Colab users)
    # from google.colab import files
    # files.download(file_name)


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64
block_size = 256
max_iters = 1000  # Reduced for faster experimentation
eval_interval = 200
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
dropout = 0.2
torch.manual_seed(1337)

file_path = '/content/input_childSpeech_trainingSet.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1)) if targets is not None else None
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

# Downsize configurations |
configs = [
    {'n_embd': 96, 'n_layer': 5, 'n_head': 4},
    # {"n_embd": 160, "n_head": 4, "n_layer": 3},
    # {"n_embd": 192, "n_head": 4, "n_layer": 2},
]

for i, config in enumerate(configs):
    n_embd = config["n_embd"]
    n_head = config["n_head"]
    n_layer = config["n_layer"]

    model = GPTLanguageModel()
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    param_count = sum(p.numel() for p in model.parameters())
    print(f"Configuration {i + 1}: {config}")
    print(f"Total Parameters: {param_count / 1e6:.6f} M")

    for iter in range(max_iters):
        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss()
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        xb, yb = get_batch('train')
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    generated_text = decode(model.generate(context, max_new_tokens=500)[0].tolist())
    file_name = f"generated_text_config_{i + 1}_params_{param_count}.txt"
    with open(file_name, "w") as f:
        f.write(generated_text)
    print(f"Generated text saved to {file_name}")


Configuration 1: {'n_embd': 96, 'n_layer': 5, 'n_head': 4}
Total Parameters: 0.590248 M
step 0: train loss 3.7647, val loss 3.7656
step 200: train loss 1.8988, val loss 1.9120
step 400: train loss 0.4246, val loss 0.4280
step 600: train loss 0.3615, val loss 0.3631
step 800: train loss 0.3506, val loss 0.3532
step 999: train loss 0.3421, val loss 0.3454
Generated text saved to generated_text_config_1_params_590248.txt
